In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida
from models.polishing import polishing
from models.filtering import filtering

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_nhanes()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.6)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (2000, 95)
33.65% missing data
Class distribution: (array([1., 2.]), array([1000, 1000]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 1),
                       (46, 9, 5),
                       (96, 11, 1),
                       (96, 13, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 4.63 sec - Loss 0.063312 - ACC 57.05% - ACC Mean 57.05% - AUC 59.01% - AUC Mean 59.01% - Deter 000
Ite 00050 - 3.27 sec - Loss 0.011090 - ACC 56.55% - ACC Mean 56.87% - AUC 57.98% - AUC Mean 59.65% - Deter 007
Ite 00100 - 1.40 sec - Loss 0.005409 - ACC 57.15% - ACC Mean 57.45% - AUC 58.82% - AUC Mean 60.17% - Deter 025
Ite 00125 - 1.64 sec - Loss 0.004318 - ACC 56.55% - ACC Mean 57.63% - AUC 59.35% - AUC Mean 60.26% - Deter 050
Early stop ite 125, rollback to correction of ite 75, whith acc of 59.2% and auc of 62.29%
OUR METHOD RUN 1/10 - acc: 59.2% +- 0.0% - auc: 62.29% +- 0.0%
Ite 00000 - 1.74 sec - Loss 0.113753 - ACC 57.70% - ACC Mean 57.70% - AUC 61.20% - AUC Mean 61.20% - Deter 000
Ite 00050 - 2.15 sec - Loss 0.038748 - ACC 57.05% - ACC Mean 57.94% - AUC 59.38% - AUC Mean 60.86% - Deter 044
Ite 00056 - 1.40 sec - Loss 0.035360 - ACC 57.70% - ACC Mean 57.84% - AUC 59.75% - AUC Mean 60.71% - Deter 050
Early stop ite 56, rollback to correction of ite 6, whith acc of 59.9

## Run MICE

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 57.65% +- 0.0% - auc: 60.8% +- 0.0%


## Run SFIL

In [6]:
sfil_accs, sfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='standard', random_state=i)
    sfil_accs.append(acc.mean())
    sfil_aucs.append(auc.mean())
    print(f'SFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SFIL GLOBAL - acc: {round(np.mean(sfil_accs) * 100, 2)}% +- {round(np.std(sfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(sfil_aucs) * 100, 2)}% +- {round(np.std(sfil_aucs) * 100, 2)}%')

SFIL RUN 1/10 - acc: 51.2% +- 2.19% - auc: 51.95% +- 2.88%
SFIL RUN 2/10 - acc: 52.5% +- 1.67% - auc: 54.18% +- 1.59%
SFIL RUN 3/10 - acc: 50.6% +- 0.34% - auc: 52.15% +- 0.95%
SFIL RUN 4/10 - acc: 51.2% +- 1.44% - auc: 52.57% +- 2.1%
SFIL RUN 5/10 - acc: 52.0% +- 0.97% - auc: 53.35% +- 2.13%
SFIL RUN 6/10 - acc: 50.45% +- 1.09% - auc: 52.59% +- 0.84%
SFIL RUN 7/10 - acc: 51.9% +- 1.04% - auc: 52.82% +- 2.0%
SFIL RUN 8/10 - acc: 52.15% +- 2.14% - auc: 52.83% +- 2.72%
SFIL RUN 9/10 - acc: 52.1% +- 3.1% - auc: 52.96% +- 4.03%
SFIL RUN 10/10 - acc: 52.05% +- 2.11% - auc: 53.12% +- 1.52%
SFIL GLOBAL - acc: 51.61% +- 0.67% - auc: 52.85% +- 0.59%


## Run PFIL

In [7]:
pfil_accs, pfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='panda', random_state=i)
    pfil_accs.append(acc.mean())
    pfil_aucs.append(auc.mean())
    print(f'PFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PFIL GLOBAL - acc: {round(np.mean(pfil_accs) * 100, 2)}% +- {round(np.std(pfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(pfil_aucs) * 100, 2)}% +- {round(np.std(pfil_aucs) * 100, 2)}%')

PFIL RUN 1/10 - acc: 55.65% +- 1.45% - auc: 58.69% +- 2.82%
PFIL RUN 2/10 - acc: 55.4% +- 1.87% - auc: 58.48% +- 1.88%
PFIL RUN 3/10 - acc: 56.6% +- 1.15% - auc: 58.44% +- 1.58%
PFIL RUN 4/10 - acc: 53.0% +- 3.02% - auc: 55.32% +- 3.11%
PFIL RUN 5/10 - acc: 54.75% +- 1.49% - auc: 57.04% +- 1.54%
PFIL RUN 6/10 - acc: 56.2% +- 1.84% - auc: 58.76% +- 2.55%
PFIL RUN 7/10 - acc: 56.3% +- 1.88% - auc: 58.54% +- 2.81%
PFIL RUN 8/10 - acc: 58.1% +- 1.97% - auc: 59.88% +- 2.81%
PFIL RUN 9/10 - acc: 54.15% +- 1.66% - auc: 57.14% +- 2.39%
PFIL RUN 10/10 - acc: 54.75% +- 1.68% - auc: 57.73% +- 2.01%
PFIL GLOBAL - acc: 55.49% +- 1.35% - auc: 58.0% +- 1.19%


## Run SPOL

In [8]:
spol_accs, spol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='standard', random_state=i)
    spol_accs.append(acc.mean())
    spol_aucs.append(auc.mean())
    print(f'SPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SPOL GLOBAL - acc: {round(np.mean(spol_accs) * 100, 2)}% +- {round(np.std(spol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(spol_aucs) * 100, 2)}% +- {round(np.std(spol_aucs) * 100, 2)}%')

SPOL RUN 1/10 - acc: 51.95% +- 2.03% - auc: 53.3% +- 2.34%
SPOL RUN 2/10 - acc: 53.45% +- 0.86% - auc: 54.27% +- 1.67%
SPOL RUN 3/10 - acc: 51.55% +- 1.35% - auc: 53.53% +- 3.18%
SPOL RUN 4/10 - acc: 50.3% +- 2.21% - auc: 50.6% +- 2.74%
SPOL RUN 5/10 - acc: 51.4% +- 1.71% - auc: 51.36% +- 2.76%
SPOL RUN 6/10 - acc: 50.35% +- 2.49% - auc: 50.7% +- 3.66%
SPOL RUN 7/10 - acc: 52.6% +- 2.3% - auc: 53.17% +- 2.46%
SPOL RUN 8/10 - acc: 52.65% +- 1.49% - auc: 53.59% +- 2.55%
SPOL RUN 9/10 - acc: 51.25% +- 2.61% - auc: 53.09% +- 3.23%
SPOL RUN 10/10 - acc: 51.95% +- 1.55% - auc: 52.68% +- 2.37%
SPOL GLOBAL - acc: 51.74% +- 0.95% - auc: 52.63% +- 1.22%


## Run PPOL

In [9]:
ppol_accs, ppol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='panda', random_state=i)
    ppol_accs.append(acc.mean())
    ppol_aucs.append(auc.mean())
    print(f'PPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PPOL GLOBAL - acc: {round(np.mean(ppol_accs) * 100, 2)}% +- {round(np.std(ppol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(ppol_aucs) * 100, 2)}% +- {round(np.std(ppol_aucs) * 100, 2)}%')

PPOL RUN 1/10 - acc: 58.9% +- 1.99% - auc: 60.75% +- 3.11%
PPOL RUN 2/10 - acc: 57.15% +- 2.58% - auc: 59.1% +- 2.66%
PPOL RUN 3/10 - acc: 56.8% +- 3.22% - auc: 59.33% +- 2.81%
PPOL RUN 4/10 - acc: 57.55% +- 1.5% - auc: 60.34% +- 1.95%
PPOL RUN 5/10 - acc: 57.2% +- 1.91% - auc: 61.71% +- 2.0%
PPOL RUN 6/10 - acc: 57.75% +- 2.56% - auc: 60.5% +- 2.98%
PPOL RUN 7/10 - acc: 58.45% +- 1.35% - auc: 61.62% +- 2.5%
PPOL RUN 8/10 - acc: 58.6% +- 2.87% - auc: 61.61% +- 2.24%
PPOL RUN 9/10 - acc: 57.05% +- 3.06% - auc: 60.12% +- 2.83%
PPOL RUN 10/10 - acc: 55.75% +- 1.08% - auc: 58.96% +- 1.42%
PPOL GLOBAL - acc: 57.52% +- 0.9% - auc: 60.4% +- 0.99%


## Run T-tests

In [10]:
for model, metrics in {
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'SFIL': {'ACC': sfil_accs, 'AUC': sfil_aucs},
        'PFIL': {'ACC': pfil_accs, 'AUC': pfil_aucs},
        'SPOL': {'ACC': spol_accs, 'AUC': spol_aucs},
        'PPOL': {'ACC': ppol_accs, 'AUC': ppol_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than SFIL
Metric AUC - OUR METHOD is significantly better than SFIL
Metric ACC - OUR METHOD is significantly better than PFIL
Metric AUC - OUR METHOD is significantly better than PFIL
Metric ACC - OUR METHOD is significantly better than SPOL
Metric AUC - OUR METHOD is significantly better than SPOL
Metric ACC - OUR METHOD is significantly better than PPOL
Metric AUC - OUR METHOD is significantly better than PPOL
